<center><h1>Semantic Similarity Analysis</h1></center>
<center><h1>Word embeddings</h1></center>

## Import librairies

In [1]:
import numpy as np
import pandas as pd
import nltk
from nltk.corpus import wordnet as wn
from nltk import word_tokenize, pos_tag
from nltk.stem import WordNetLemmatizer
import spacy
import transformers
from transformers import pipeline
from transformers import logging
import warnings
logging.set_verbosity_error()
warnings.filterwarnings('ignore')

## Import Datasets

In [2]:
tm_df = pd.read_excel("../../Datasets/text_mining_results_ngrams.xlsx")
tm_df.head()

,target sentence,noisy sentence1,noisy sentence2,noisy sentence3,common_sentence,uncommon_target,uncommon_sentence1,uncommon_sentence2,uncommon_sentence3,Is correct
0,A man with a hard hat is dancing.,A CAN WHET a GERARD hat is dancing.,A man with a hard hat is dancing.,A CAN WHET IO GER ARD AT FRIZZ DUN cing.,a [MASK] [MASK] [MASK] [MASK] [MASK] [MASK] [M...,"['man', 'with', 'a', 'hard', 'hat', 'is', 'dan...","['can', 'whet', 'a', 'gerard', 'hat', 'is', 'd...","['man', 'with', 'a', 'hard', 'hat', 'is', 'dan...","['can whet', 'io', 'ger', 'ard at', 'frizz', '...",Correct
1,A young child is riding a horse.,A young child is REDDING a horse.,A young TILED WIZ riding IO horse.,A YOUNG CHILLED HIS RED DING AI horse.,a young [MASK] [MASK] [MASK] [MASK] horse,"['child', 'is', 'riding', 'a']","['child', 'is', 'redding', 'a']","['tiled', 'wiz', 'riding', 'io']","['chilled his', 'red', 'ding', 'ai']",Correct
2,A man is feeding a mouse to a snake.,A man is feeding IO mouse to IO snake.,A CAN is feeding a mouse to a snake.,A CAN BIZ SUPERSEDE MING AWE HOUSE ADO AWE snake.,a [MASK] [MASK] [MASK] [MASK] [MASK] [MASK] [M...,"['man', 'is', 'feeding', 'a', 'mouse', 'to', 'a']","['man', 'is', 'feeding', 'io', 'mouse', 'to', ...","['can', 'is', 'feeding', 'a', 'mouse', 'to', 'a']","['can', 'biz supersede', 'ming', 'awe', 'house...",Correct
3,A woman is playing the guitar.,A woman is playing the guitar.,A WAYMAN WIZ SWAYING TU guitar.,A WAY MAN WIZ SWAY ING TU GOY tar.,a [MASK] [MASK] [MASK] [MASK] [MASK],"['woman', 'is', 'playing', 'the', 'guitar']","['woman', 'is', 'playing', 'the', 'guitar']","['wayman', 'wiz', 'swaying', 'tu', 'guitar']","['way', 'man wiz', 'sway', 'ing tu', 'goy tar']",Correct
4,A woman is playing the flute.,A WAYMAN WIZ SWAYING TU flute.,A woman is playing the flute.,A WEY MUN ICH WAY JING TOW flute.,a [MASK] [MASK] [MASK] [MASK] flute,"['woman', 'is', 'playing', 'the']","['wayman', 'wiz', 'swaying', 'tu']","['woman', 'is', 'playing', 'the']","['wey', 'mun', 'ich way', 'jing tow']",Correct


In [3]:
db_df = pd.read_excel("../../Datasets/distilbert-base-uncased_pred_ngram.xlsx")
db_df.head()

,Masked sentence,real,pred
0,a [MASK] [MASK] [MASK] [MASK] [MASK] [MASK] [M...,man,"['##quil', 'single', '.', 'pseudo', '##yt']"
1,a man [MASK] [MASK] [MASK] [MASK] [MASK] [MASK],with,"['whose', 'who', 'called', 'named', 'with']"
2,a man with [MASK] [MASK] [MASK] [MASK] [MASK],a,"['a', 'no', 'an', 'his', 'the']"
3,a man with a [MASK] [MASK] [MASK] [MASK],hard,"['thin', 'red', 'blue', 'large', 'thick']"
4,a man with a hard [MASK] [MASK] [MASK],hat,"['-', 'jaw', 'left', 'iron', 'tooth']"


## Text Preprocessing

In [4]:
def tokenize(sentence):
    return nltk.RegexpTokenizer(r'\w+').tokenize(sentence)

In [9]:
masked_sentences = tm_df['common_sentence'].tolist()
preds = db_df["pred"].apply(lambda x: tokenize(x)).tolist()
uncommon_parts1 = tm_df['uncommon_sentence1'].apply(lambda x: tokenize(x)).tolist()
uncommon_parts2 = tm_df['uncommon_sentence2'].apply(lambda x: tokenize(x)).tolist()
uncommon_parts3 = tm_df['uncommon_sentence3'].apply(lambda x: tokenize(x)).tolist()

## Semantic Similarity Analysis with Word Embedings

In [6]:
nlp = spacy.load("en_core_web_md")

In [ ]:
j = 0
final_results = []
final_sentences = []
for i in range(len(masked_sentences)):
    print(i+1, "/", len(masked_sentences))
    ms = masked_sentences[i]
    fs = ms
    uncommon_part1 = uncommon_parts1[i]
    uncommon_part2 = uncommon_parts2[i]
    uncommon_part3 = uncommon_parts3[i]

    for l in range(min(len(uncommon_part1), len(uncommon_part2), len(uncommon_part3))):
        strings = [uncommon_part1[l], uncommon_part2[l], uncommon_part3[l]]
        similarity = []
        for s in strings:
            similarity.append(np.mean([nlp(w).similarity(nlp(s)) for w in preds[j]]))
        final_results.append([ms, strings[0], strings[1], preds[j], similarity[0], similarity[1], strings[np.argmax(similarity)]])
        j += 1
        fs = fs.replace("[MASK]", strings[np.argmax(similarity)], 1)
    final_sentences.append(fs)

In [11]:
final_results = pd.DataFrame(final_results, columns=["masked_sentence", "uncommon_part1", "uncommon_part2", "pred", "similarity1", "similarity2", "final_result"])
final_results

,masked_sentence,uncommon_part1,uncommon_part2,pred,similarity1,similarity2,final_result
0,a [MASK] [MASK] [MASK] [MASK] [MASK] [MASK] [M...,can,man,"[quil, single, pseudo, yt]",0.137789,0.093315,can
1,a [MASK] [MASK] [MASK] [MASK] [MASK] [MASK] [M...,whet,with,"[whose, who, called, named, with]",0.053405,0.405943,with
2,a [MASK] [MASK] [MASK] [MASK] [MASK] [MASK] [M...,a,a,"[a, no, an, his, the]",0.456967,0.456967,a
3,a [MASK] [MASK] [MASK] [MASK] [MASK] [MASK] [M...,gerard,hard,"[thin, red, blue, large, thick]",-0.095775,0.228202,hard
4,a [MASK] [MASK] [MASK] [MASK] [MASK] [MASK] [M...,hat,hat,"[jaw, left, iron, tooth]",0.099214,0.099214,hat
...,...,...,...,...,...,...,...
12497,but [MASK] [MASK] [MASK] [MASK] [MASK] [MASK] ...,maturity,security,"[and, or, hyper]",0.264798,0.263510,maturity
12498,but [MASK] [MASK] [MASK] [MASK] [MASK] [MASK] ...,features,features,"[and, or, of, in]",0.268211,0.268211,features
12499,but [MASK] [MASK] [MASK] [MASK] [MASK] [MASK] ...,misconstrue,to,"[loss, deficiency, gain, bearing, lessness]",0.199063,0.186555,misconstrue
12500,but [MASK] [MASK] [MASK] [MASK] [MASK] [MASK] ...,set,set,"[syndrome, symptoms, disorder, hyper, and]",0.077810,0.077810,ease


In [13]:
final_sentences = pd.DataFrame(final_sentences, columns=["final_sentence"])
final_sentences.insert(0, "target_sentence", tm_df["target sentence"].tolist()[:1240])
final_sentences

,target_sentence,final_sentence
0,A man with a hard hat is dancing.,a can with a hard hat is dancing
1,A young child is riding a horse.,a young child is riding a horse
2,A man is feeding a mouse to a snake.,a man is supersede a mouse to a snake
3,A woman is playing the guitar.,a way man swaying the guitar
4,A woman is playing the flute.,a woman is playing the flute
...,...,...
1235,"These men ""are entitled to respect for their p...",these men are entitled to respect wherefore th...
1236,"The mock explosion, the first event in the dri...",the mosque explosion the first event in the dr...
1237,Aiken's study appears in the Sept. 24 issue of...,aikens study appears in thereof the 24 issue t...
1238,"A Merrill Lynch spokesman said ""we are pleased...",a merrill lynch spokesman tan we the with plac...


In [68]:
pd.DataFrame(final_results).to_excel("../../Datasets/Word_Embedding_Results.xlsx", index=False)
pd.DataFrame(final_sentences).to_excel("../../Datasets/eval_dataset_pipeline1_bis.xlsx", index=False)